In [1]:
import json
import pandas as pd
import re
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, LSTM, Embedding

from keras.preprocessing import sequence
from keras.utils import np_utils

import os
import shutil
import urllib3
import zipfile
import pandas as pd
import json

In [7]:
f = open("[라벨]경상도_학습데이터_1/DKCI20000001.json", encoding="UTF-8")
raw_data = json.loads(f.read())

In [8]:
# 사투리, 표준어 사전 생성 함수
def make_dict(raw_data):

    dialect_word = []
    standard_word = [] 
    
    
    for i in range(len(raw_data["utterance"])):
        eojeol = raw_data["utterance"][i]["eojeolList"]
            
        for k in range(len(eojeol)):
            if eojeol[k]["isDialect"] == True:
                if k not in dialect_word:
                    dialect_word.append(eojeol[k]['eojeol'])
                    standard_word.append(eojeol[k]['standard'])

    word_dictionary = pd.DataFrame([dialect_word, standard_word])
    word_dictionary = word_dictionary.transpose()
    word_dictionary.columns = ['dialect', 'standard']
    dialect_dictionary = word_dictionary['dialect']
    standard_dictionary = word_dictionary['standard']
    dialect_dict = dialect_dictionary.to_dict()
    standard_dict = standard_dictionary.to_dict()

    index_to_dialect = {v:k for k,v in dialect_dict.items()}
    index_to_standard = {v:k for k,v in standard_dict.items()}

    dialect_to_index = {v:k for k,v in index_to_dialect.items()}
    standard_to_index = {v:k for k,v in index_to_standard.items()}

    print(len(index_to_dialect))
    print(len(dialect_to_index))

    print(len(index_to_standard))
    print(len(standard_to_index))
    

    return dialect_to_index,standard_to_index,index_to_dialect,index_to_standard
   

In [10]:
index_to_dialect,index_to_standard,dialect_to_index,standard_to_index = make_dict(raw_data)

236
236
223
223


In [11]:
# 사투리 표준어 쌍 생성 함수
def make_pair(raw_data):
    filter = re.compile('[^가-힣+]')

    dialect_form = []
    standard_form = []

    for i in range(len(raw_data["utterance"])):
        sentence = raw_data["utterance"][i]
        
        for j in range(len(sentence["eojeolList"])):
            #print(sentence["eojeolList"][0]['eojeol'])
            if sentence["eojeolList"][j]['isDialect'] == True:
                try : 

                    dialect = sentence["eojeolList"][j]["eojeol"]
                    standard = sentence["eojeolList"][j]["standard"]
                    
                    dialect = filter.sub('', dialect)
                    standard = filter.sub('', standard)
                    
                    dialect = dialect_to_index[dialect]
                    standard = standard_to_index[standard]

                    dialect_form.append(dialect)
                    standard_form.append(standard)
                except:
                    pass

    result = pd.DataFrame(data = {"dialect_words":dialect_form, "standard_words": standard_form})


    return result
               

In [12]:
pairs = make_pair(raw_data)

In [39]:
# 사투리 사전에 존재하는지 확인하는 함수

def exist_in_dialect(x):

    if x in dialect_to_index:
        return True
    else:
        return False


In [40]:
exist_in_dialect("자가격리")

False

In [14]:
xtrain = np_utils.to_categorical(pairs["dialect_words"])
ytrain = np_utils.to_categorical(pairs["standard_words"])
len(xtrain[0])

416

In [15]:
xtrain.shape

(398, 416)

In [16]:
model = Sequential()
model.add(Dense(256, input_dim = 416, activation="relu"))
model.add(Dense(416, activation="softmax"))

In [17]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [18]:
model.fit(xtrain, ytrain, batch_size=100, epochs=100)

Epoch 1/100
4/4 [==============================] - 1s 7ms/step - loss: 6.0239 - accuracy: 0.0000e+00
Epoch 2/100
4/4 [==============================] - 0s 8ms/step - loss: 5.9675 - accuracy: 0.2060
Epoch 3/100
4/4 [==============================] - 0s 7ms/step - loss: 5.9117 - accuracy: 0.4221
Epoch 4/100
4/4 [==============================] - 0s 11ms/step - loss: 5.8536 - accuracy: 0.5176
Epoch 5/100
4/4 [==============================] - 0s 16ms/step - loss: 5.7904 - accuracy: 0.5754
Epoch 6/100
4/4 [==============================] - 0s 14ms/step - loss: 5.7200 - accuracy: 0.5905
Epoch 7/100
4/4 [==============================] - 0s 16ms/step - loss: 5.6429 - accuracy: 0.5980
Epoch 8/100
4/4 [==============================] - 0s 15ms/step - loss: 5.5548 - accuracy: 0.5955
Epoch 9/100
4/4 [==============================] - 0s 15ms/step - loss: 5.4581 - accuracy: 0.5804
Epoch 10/100
4/4 [==============================] - 0s 13ms/step - loss: 5.3483 - accuracy: 0.5628
Epoch 11/100
4/4 [

In [24]:
# 문장 번역 함수
def translate(sentence):
    res = ""

    sentence = sentence.split()

    for i in range(len(sentence)):
        if exist_in_dialect(sentence[i]) == True:
            print(dialect_to_index[sentence[i]])
            index = dialect_to_index[sentence[i]]
            one_hot = np_utils.to_categorical(index,len(xtrain[0])) #인덱스 배열의크기 지정 필요 size_of_dialect
            pred_index = np.argmax(model.predict(np.array([one_hot])))
            res = res + " " + index_to_standard[pred_index]

        else:
            res = res + " " + sentence[i]

    return res


In [25]:
translate("자가격리 이주 했다 아이가")

KeyError: '자가격리'

In [30]:
exist_in_dialect("인덱스")

True

In [ ]:
pos_dict = {"NNG": 0, "NNP": 1, "NNB": 2, "NP": 3, "NR": 4, "VV": 5,
           "VA": 6, "VX": 7, "VCP": 8, "VCN": 9, "MM": 10, "MAG": 11,
           "MAJ": 12, "IC": 13, "JKS": 14, "JKC": 15, "JKG": 16, "JKO": 17,
           "JKB": 18, "JKV": 19, "JKQ": 20, "JX": 21, "JC": 22, "EP": 23,
           "EF": 24, "EC": 25, "ETN": 26, "ETM": 27, "XPN": 28, "XSN": 29,
           "XSV": 30, "XSA": 31, "XR": 32, "SF":  33, "SP": 34, "SS": 35,
           "SE": 36, "SO": 37,  "SL": 38, "SH": 39, "SW": 40, "NF": 41,
           "NV": 42, "SN": 43, "NA": 44}

In [2]:
def make_eojeol(standard_to_index, dialect_to_index):
    eojeol_list=[]
    st_list = []
    di_list = []
    
    for k in standard_to_index:
        st_list.append(k)
        
    for v in dialect_to_index:
        di_list.append(v)
        
    for i, j in zip(st_list, di_list):
        
        eojeol_list.append(i)
        
        if i == j:
            eojeol_list.append(j + "0")
        else:
            eojeol_list.append(j)   
            
    eojeol_df = pd.DataFrame(eojeol_list, columns=None)
    eojeol_dict = eojeol_df.to_dict()
    eojeol_dict = eojeol_dict[0]
    eojeol_dict = {v:k for k,v in eojeol_dict.items()}
    print(eojeol_dict)

    return eojeol_dict

In [ ]:
eojeol_dict = make_eojeol()


In [3]:
def eojeol_dependence():
    for i in range(len(raw_data["utterance"])):
        sta=raw_data['utterance'][i]['standard_form']
        dia=raw_data['utterance'][i]['dialect_form']
        print(sta.split())

In [ ]:
df_eojeol_dependence = eojeol_dependence()

In [4]:
def predict_synonym():
    res = 0
    return res